In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import requests
import json
import plotly.graph_objects as go
import plotly.express as px
import nltk
import re
from unicodedata import normalize
from nltk.corpus import stopwords 
pd.set_option('display.max_colwidth', None)

In [ ]:
# Esta linea es importante para descargar el archivo con las stopwords en el path indicado
# Una vez descargado ya se puede comentar o eliminar la linea
# La dejo a modo de documentación 

# nltk.download('stopwords', download_dir='/home/agu/Desktop/tesis/stopwords')

In [ ]:
# Con esta linea se agrega el path en el que se descargaron las stopwords
# Es importante ya que sino no va a encontrar el archivo

nltk.data.path.append("/home/agu/Desktop/tesis/stopwords") 

In [ ]:
def convert_response_to_list(reviews):
    list_reviews = []
    for review in reviews:
        new_json = {
            "titulo": review['title'],
            "comentario": review['content'].lower(),
            "valoracion": review['rate']
        }
        
        list_reviews.append(new_json)
    return list_reviews

def obtener_df_reviews(idProd):
    url_api = "https://api.mercadolibre.com/reviews/item/"
    url = url_api + idProd
    args = {'limit': 200}

    response = requests.get(url, params=args)
    if response.status_code != 200:
        return ""
    response_json = json.loads(response.text)
    reviews = convert_response_to_list(response_json['reviews'])
    df = pd.DataFrame.from_records(reviews)
    return df

def preprocesar_comentarios_df(df):
    # Elimino signos como guines, dos puntos, etc.
    df["comentario"] = df['comentario'].str.replace('[^\w\s]','')
    
    # Dejo solo las palabras que no se encuentren dentro de las stopwords y que no son numericas
    df["comentario"] = df["comentario"].apply(
        lambda x: ' '.join(
            [word for word in x.split() if (word.lower() not in (stop)) and (word.isnumeric() == False)]
        )
    )
    
    # Con estas sentencias elimino todo tilpo de tildes, dieresis, etc (a excepcion de las ñ)
    df["comentario"] = df["comentario"].apply(
        lambda x: normalize(
            'NFC', 
             re.sub(
                r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
                normalize("NFD", x), 0, re.I
            )                  
       )
    )
    
    return df[["comentario"]]

def contar_frecuencia_palabras_df(df):
    # Cuento la frecuencia con la que aparece cada palabra
    df_frecuencia = df.comentario.str.split(expand=True).stack().value_counts().reset_index()
    df_frecuencia.columns = ['palabra', 'frecuencia']
    return df_frecuencia

def plotear_frecuencia_palabras(df, tope_palabras, producto):
    fig = px.bar(
        df[0:tope_palabras], 
        x='palabra', 
        y='frecuencia',
        title = f'Top 50 palabras con mayor frecuencia del producto "{producto}"',
        labels = {
            "frecuencia": "Cantidad de veces que aparece la palabra",
            "palabra": "Palabra"
        }
    )
    fig.show()

In [ ]:
df_prod_1 = obtener_df_reviews("MLA17464694") # Samsung A02
df_prod_2 = obtener_df_reviews("MLA17415925") # Samsung A12
df_prod_3 = obtener_df_reviews("MLA17706115") # Samsung A32

# 1er Análisis: Relación entre la valoración del comentario y el contenido

In [ ]:
df_prod_1.iloc[[1,28,31,58,126,141,60,47,129,12]]

In [ ]:
df_prod_2.iloc[[146,94,85,178,199,24,16,1,28,79,57]]

In [ ]:
df_prod_3.iloc[[100,69,59,7,189,171,56,22,10,60,42,3,12,112]]

In [ ]:
porc_prod_1 = round((len(df_prod_1.iloc[[11,28,31,58,126,141,60,47,129,12]]) / len(df_prod_1)) * 100, 2)
porc_prod_2 = round((len(df_prod_2.iloc[[146,94,85,178,199,24,16,1,28,79,57]]) / len(df_prod_2)) * 100, 2)
porc_prod_3 = round((len(df_prod_3.iloc[[100,69,59,7,189,171,56,22,10,60,42,3,12,112]]) / len(df_prod_3)) * 100, 2)

In [ ]:
productos = ['Samsung A02', 'Samsung A12', 'Samsung A32']

fig = go.Figure([go.Bar(x=productos, y=[porc_prod_1, porc_prod_2, porc_prod_3])])

fig.update_yaxes(
    title = "Porcentaje"
)

fig.update_layout(
    title = "Porcentaje de comentarios con valoración errónea",
)

fig.show()

# 2do Análisis

In [ ]:
df_prod_1_positivos = df_prod_1[df_prod_1["comentario"].str.contains("bueno")]
df_prod_1_negativos = df_prod_1[df_prod_1["comentario"].str.contains("malo")]
porc_comentarios_positivos_prod_1 = (len(df_prod_1_positivos) / len(df_prod_1)) * 100
porc_comentarios_negativos_prod_1 = (len(df_prod_1_negativos) / len(df_prod_1)) * 100


df_prod_2_positivos = df_prod_2[df_prod_2["comentario"].str.contains("bueno")]
df_prod_2_negativos = df_prod_2[df_prod_2["comentario"].str.contains("malo")]
porc_comentarios_positivos_prod_2 = (len(df_prod_2_positivos) / len(df_prod_2)) * 100
porc_comentarios_negativos_prod_2 = (len(df_prod_2_negativos) / len(df_prod_2)) * 100


df_prod_3_positivos = df_prod_3[df_prod_3["comentario"].str.contains("bueno")]
df_prod_3_negativos = df_prod_3[df_prod_3["comentario"].str.contains("malo")]
porc_comentarios_positivos_prod_3 = (len(df_prod_3_positivos) / len(df_prod_3)) * 100
porc_comentarios_negativos_prod_3 = (len(df_prod_3_negativos) / len(df_prod_3)) * 100

In [ ]:
productos = ['Samsung A02', 'Samsung A12', 'Samsung A32']

fig = go.Figure(data=[
    go.Bar(name='Positivos', x = productos, y=[porc_comentarios_positivos_prod_1, porc_comentarios_positivos_prod_2, porc_comentarios_positivos_prod_3]),
    go.Bar(name='Negativos', x = productos, y=[porc_comentarios_negativos_prod_1, porc_comentarios_negativos_prod_2, porc_comentarios_negativos_prod_3])
])

fig.update_yaxes(
    title = "Porcentaje"
)

fig.update_layout(
    title = "Porcentaje de comentarios POSITIVOS y NEGATIVOS por Producto",
)

fig.show()

## Algunos Ejemplos

### Samsung A02

In [ ]:
df_prod_1_positivos.head(5)

In [ ]:
df_prod_1_negativos

### Samsung A12

In [ ]:
df_prod_2_positivos.head(5)

In [ ]:
df_prod_2_negativos

### Samsung A32

In [ ]:
df_prod_3_positivos.head(5)

In [ ]:
df_prod_3_negativos

# 3. ¿Cual es la distribución de palabras mas frecuentemente utilizada?

In [ ]:
stop = stopwords.words('spanish')

In [ ]:
df_prod_1_procesado = preprocesar_comentarios_df(df_prod_1)
df_prod_2_procesado = preprocesar_comentarios_df(df_prod_2)
df_prod_3_procesado = preprocesar_comentarios_df(df_prod_3)

In [ ]:
frecuencia_palabras_prod_1 = contar_frecuencia_palabras_df(df_prod_1_procesado)
frecuencia_palabras_prod_2 = contar_frecuencia_palabras_df(df_prod_2_procesado)
frecuencia_palabras_prod_3 = contar_frecuencia_palabras_df(df_prod_3_procesado)

In [ ]:
plotear_frecuencia_palabras(frecuencia_palabras_prod_1, 50, "Samsung A02")
plotear_frecuencia_palabras(frecuencia_palabras_prod_2, 50, "Samsung A12")
plotear_frecuencia_palabras(frecuencia_palabras_prod_3, 50, "Samsung A32")